In [1]:
from datetime import datetime
from zoneinfo import ZoneInfo

LONDON = ZoneInfo("Europe/London")
now = datetime.now(LONDON)
print(now, now.tzinfo)

2026-02-02 14:18:00.691509+00:00 Europe/London


In [2]:
import __main__
from masphd.models.transformers import ColumnDropper
__main__.ColumnDropper = ColumnDropper


from datetime import datetime
from zoneinfo import ZoneInfo

from masphd.darwin.extract_segments import extract_segments
from masphd.darwin.realtime_filter import filter_segments_by_now
from masphd.features.segment_features import SegmentFeatureBuilder
from masphd.models.ensemble import WeightedEnsemblePredictor

import pandas as pd

LONDON = ZoneInfo("Europe/London")

feature_builder = SegmentFeatureBuilder(tz=LONDON)
ensemble = WeightedEnsemblePredictor(weights_filename="model_weights.json")

def on_decoded(forecasts, schedules, xml_bytes):
    now = datetime.now(LONDON)

    segs = extract_segments(forecasts, schedules, tz=LONDON, drop_wrong_direction=True)

    # planned arrival at SECOND station for in-progress filtering
    for s in segs:
        loc_second = s.get("loc_second") or {}
        s["planned_arr_second"] = loc_second.get("pta") or loc_second.get("wta")

    segs = filter_segments_by_now(
        segs,
        now=now,
        tz=LONDON,
        mode="in_progress",
        dep_grace_after_now_mins=5,
        arr_grace_before_now_mins=2,
    )

    if not segs:
        return

    results = []
    for s in segs:
        feat = feature_builder.build(s)
        if feat is None:
            continue

        # IMPORTANT:
        # Most sklearn pipelines expect a DataFrame, so use single-row DataFrame
        X = pd.DataFrame([feat])

        yhat = ensemble.predict_one(s["first"], s["second"], X)
        if yhat is None:
            continue

        results.append((s, feat, yhat))

    if not results:
        return

    actual_group = [r for r in results if r[0].get("has_actual_dep")]
    other_group = [r for r in results if not r[0].get("has_actual_dep")]

    # print("\n=== PREDICTIONS (in progress) ===", now.strftime("%Y-%m-%d %H:%M:%S"))
    # print(f"Total predicted: {len(results)} | actual_dep: {len(actual_group)} | other: {len(other_group)}")

    # print(schedules)
    # print(forecasts)

    if actual_group:
        print("\n--- CONFIRMED ACTUAL DEPARTURE (save later) ---")
        for s, feat, yhat in actual_group:
            print(
                f'{s["first"]}->{s["second"]} | dep_kind={s["dep_time_kind"]} '
                f'planned_dep={s["planned_dep"]} actual_dep={s["actual_dep_confirmed"]} '
                f'| dep_delay={feat["departure_delay"]:.1f} dwell={feat["dwell_delay"]:.1f} '
                f'| pred_arrival_delay_min={yhat:.2f}'
            )

    # if other_group:
    #     print("\n--- ESTIMATE / NO CONFIRMED ACTUAL (show separately) ---")
    #     for s, feat, yhat in other_group:
    #         print(
    #             f'{s["first"]}->{s["second"]} | dep_kind={s["dep_time_kind"]} '
    #             f'planned_dep={s["planned_dep"]} best_dep={s["dep_time_for_prediction"]} '
    #             f'| dep_delay={feat["departure_delay"]:.1f} dwell={feat["dwell_delay"]:.1f} '
    #             f'| pred_arrival_delay_min={yhat:.2f}'
    #         )
    print("*" * 20)

In [3]:
from masphd.darwin.client import DarwinClient

client = DarwinClient(on_decoded=on_decoded)
client.connect()
client.run_for(300)



=== PREDICTIONS (in progress) === 2026-02-02 14:18:26
Total predicted: 1 | actual_dep: 0 | other: 1
********************

=== PREDICTIONS (in progress) === 2026-02-02 14:18:27
Total predicted: 1 | actual_dep: 0 | other: 1
********************

=== PREDICTIONS (in progress) === 2026-02-02 14:18:28
Total predicted: 1 | actual_dep: 0 | other: 1
********************

=== PREDICTIONS (in progress) === 2026-02-02 14:18:28
Total predicted: 2 | actual_dep: 0 | other: 2
********************

=== PREDICTIONS (in progress) === 2026-02-02 14:18:34
Total predicted: 1 | actual_dep: 0 | other: 1
********************

=== PREDICTIONS (in progress) === 2026-02-02 14:18:35
Total predicted: 1 | actual_dep: 0 | other: 1
********************

=== PREDICTIONS (in progress) === 2026-02-02 14:18:40
Total predicted: 1 | actual_dep: 0 | other: 1
********************

=== PREDICTIONS (in progress) === 2026-02-02 14:18:44
Total predicted: 1 | actual_dep: 0 | other: 1
********************

=== PREDICTIONS (in pro

Disconnected. Sleeping 15 seconds.
